# maimai 谱面难度预测 - 基于 LSTM 的时序建模

本项目使用 LSTM 神经网络直接处理谱面的 note 序列数据，将每个 note 的时间戳和类型等信息作为时序特征输入模型，预测谱面的难度定数。

**核心思路**：将谱面视为时间序列，每个 note 包含时间戳、类型、位置等属性，通过 LSTM 学习 note 序列的时序特征来预测难度。

## 1. 导入所需库

In [42]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import csv
import os
import sys

## 2. 数据处理与序列化

数据处理分为两个主要步骤：
1. **谱面解析**：将 maidata.txt 格式解析为结构化的 note 序列数据
2. **序列预处理**：将 note 序列转换为适合 LSTM 输入的格式

**核心理念**：每个谱面是一个时间序列，包含按时间顺序排列的 note 序列。每个 note 具有时间戳、类型、位置等属性。

### 2.1 解析 maidata.txt

我们使用外部工具 `SimaiSerializerFromMajdataEdit.exe` 来将 `maidata.txt` 格式的谱面文件解析并序列化为 JSON 文件。

数据来源：maichart-converts

**使用方法:**

在终端中执行以下命令，它会将 `data\maichart-converts` 目录下的所有谱面处理并输出到 `data\serialized` 目录。


In [43]:
command = (
    r"src\serializer\src\bin\Release\net8.0\SimaiSerializerFromMajdataEdit.exe "
    r"data\maichart-converts data\serialized"
)
print(command)

src\serializer\src\bin\Release\net8.0\SimaiSerializerFromMajdataEdit.exe data\maichart-converts data\serialized


该工具的通用命令格式为： `SimaiSerializerFromMajdataEdit.exe <输入文件或目录> <输出目录>`

执行完毕后，我们将得到包含 note 序列数据的 JSON 文件，每个文件对应一个特定难度的谱面。

**TODO**：
- 运行序列化工具并检查输出结果
- 验证生成的 JSON 文件结构
- 统计不同谱面的 note 数量分布，为序列长度标准化做准备


### 2.2 处理谱面标签数据

从 maimai-songs 库的 songs.json 中提取训练标签：
- **歌曲ID**：song_id（json中为id）
- **难度序号**：level_index（在json中并未显式标明，charts中依次对应level_index 1-5的数据）
- **难度定数**：difficulty_constant（json中为level）- 这是我们的预测目标

**TODO**：
- 提取标签数据并与序列化的谱面数据进行匹配
- 处理缺失的难度定数（null值）
- 过滤掉六位数ID的宴谱数据
- 从 flevel.json 中获取拟合等级数据作为辅助信息
- 验证标签与谱面文件的一一对应关系

In [44]:
def extract_and_write_song_info_with_json(serialized_dir, songs_metadata_path, csv_file_path):
    """
    1. 解析 songs.json，提取 (song_id, level_index, difficulty_constant)
    2. 查找对应的 serialized json 文件，写入 json_filename
    3. 只保留有 json 文件的条目，一次性写入 CSV
    """
    import glob
    # 读取JSON文件
    if not os.path.exists(songs_metadata_path):
        print(f"错误：文件不存在 - {songs_metadata_path}")
        sys.exit(1)
    with open(songs_metadata_path, 'r', encoding='utf-8') as f:
        songs_data = json.load(f)

    # 建立 (song_id, level_index) -> json_filename 映射
    json_files = glob.glob(os.path.join(serialized_dir, "*.json"))
    json_map = {}
    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as jf:
                data = json.load(jf)
                song_id = int(data['song_id'])
                level_index = int(data['level_index'])
                json_map[(song_id, level_index)] = os.path.basename(json_file)
        except Exception as e:
            print(f"解析失败: {json_file}, 错误: {e}")
            continue

    # 提取所需信息并查找json文件名，只保留有json文件的条目
    extracted_info = []
    for song in songs_data:
        song_id = song.get('id')
        charts = song.get('charts', [])
        for level_index, chart in enumerate(charts, start=1):
            difficulty_constant = chart.get('level')
            try:
                sid = int(song_id)
                lid = int(level_index)
            except Exception:
                continue
            json_filename = json_map.get((sid, lid))
            if json_filename is not None:
                extracted_info.append({
                    'song_id': sid,
                    'level_index': lid,
                    'difficulty_constant': difficulty_constant,
                    'json_filename': json_filename
                })

    # 写入CSV
    os.makedirs(os.path.dirname(csv_file_path), exist_ok=True)
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
        fieldnames = ['song_id', 'level_index', 'difficulty_constant', 'json_filename']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for item in extracted_info:
            writer.writerow(item)
    print(f"成功提取 {len(extracted_info)} 条记录（均有json文件），已写入 {csv_file_path}")

# 用法示例
# base_dir = os.path.dirname(os.path.abspath(''))
# serialized_dir = os.path.join(base_dir, "data", "serialized")
# songs_json_path = os.path.join(base_dir, "data", "maimai-songs", "songs.json")
# csv_path = os.path.join(base_dir, "data", "song_info.csv")
# extract_and_write_song_info_with_json(serialized_dir, songs_json_path, csv_path)

### 2.3 划分训练集与测试集

我们需要排除宴谱，然后：
- 对绿/黄/红/紫白谱分别进行训练集/测试集划分，保证每组比例相同
- 可能甚至要将紫白谱的每个定数都保证内部均分
- 允许将特定谱面排除在训练集之外

## 3. 序列预处理与特征编码

不同于传统的特征工程方法，我们直接使用原始的 note 序列数据。主要任务是将 note 属性转换为数值向量，并处理序列长度不一致的问题。

### 3.1 构建自定义Dataset类
我们创建一个自定义Dataset，存储json文件的位置以及csv的位置。
在Dataset中，需要实现：
1. `__init__()`：初始化函数，传入serialized目录以及csv文件位置。
    - 需要存储每个json的路径
2. `__len__()`：返回数据集的长度。
3. `__getitem__()`：返回数据集中的第i个样本。直接返回tensor
    - 在`__getitem__()`中才读取json文件，并返回tensor
    - 读取json文件，然后再去csv中找对应`(song_id,level_index)`的行
    - index顺序是什么

In [45]:
class MaichartDataset(Dataset):
    def __init__(self, serialized_dir, labels_csv):
        self.serialized_dir = serialized_dir
        
        # 读取CSV并清理数据
        self.labels_data = pd.read_csv(labels_csv)
        self.labels_data = self.labels_data.dropna(subset=['song_id', 'level_index', 'difficulty_constant', 'json_filename'])
        
        # 重置索引以确保连续的整数索引
        self.labels_data = self.labels_data.reset_index(drop=True)

        # TouchArea映射
        self.touch_area_mapping = {" ": 0, "A": 1, "D": 2, "E": 3, "B": 4, "C": 5} # 从外到内

        # 初始化编码器
        self._setup_encoders()

    def _setup_encoders(self):
        """设置note类型和位置的编码器"""
        # Note类型编码器
        self.NOTE_TYPES = ['Tap', 'Hold', 'Slide', 'Touch', 'TouchHold']
        self.note_type_encoder = OneHotEncoder(
            sparse_output=False,
            dtype=np.float32,
            handle_unknown='ignore'
        )
        self.note_type_encoder.fit(np.array(self.NOTE_TYPES).reshape(-1, 1))
        
        # 位置编码器（假设位置范围是1-8）
        self.positions = list(range(1, 9))  # maimai有8个位置
        self.position_encoder = OneHotEncoder(
            sparse_output=False,
            dtype=np.float32,
            handle_unknown='ignore'
        )
        self.position_encoder.fit(np.array(self.positions).reshape(-1, 1))

    def _extract_note_features(self, note, time):
        """
        从单个note中提取特征向量
        
        Args:
            note: 包含note信息的字典
            time: note的时间戳
            
        Returns:
            np.ndarray: 21维的特征向量
        """
        # 编码note类型和位置
        note_type_encoded = self.note_type_encoder.transform([[note['noteType']]])[0]
        position_encoded = self.position_encoder.transform([[note['startPosition']]])[0]
        
        # 提取其他特征
        hold_time = note.get('holdTime', 0)
        is_break = int(note['isBreak'])
        is_ex = int(note['isEx'])
        is_slide_break = int(note['isSlideBreak'])
        slide_start_time = note['slideStartTime']
        slide_end_time = slide_start_time + note['slideTime']
        touch_area = self.touch_area_mapping[note['touchArea']]
        
        # 组合特征向量
        feature_vector = np.concatenate([
            [time],             # 1维
            note_type_encoded,  # 5维
            position_encoded,   # 8维
            [hold_time],        # 1维
            [is_break],         # 1维
            [is_ex],            # 1维
            [is_slide_break],   # 1维
            [slide_start_time], # 1维
            [slide_end_time],   # 1维
            [touch_area]        # 1维
        ])  # 总共 21维
        
        return feature_vector

    def _extract_sequence_features(self, json_data):
        """
        从JSON数据中提取整个谱面的note序列特征
        
        Args:
            json_data: 包含谱面数据的JSON对象
            
        Returns:
            list: note特征向量的列表
        """
        note_groups = json_data.get('notes', [])
        note_features_sequence = []
        
        for note_group in note_groups:
            time = note_group['Time']
            notes = note_group['Notes']
            
            for note in notes:
                feature_vector = self._extract_note_features(note, time)
                note_features_sequence.append(feature_vector)
        
        return note_features_sequence

    def _extract_sequence_features_vectorized(self, json_data):
        """
        向量化提取整个谱面的note序列特征
        
        Args:
            json_data: 包含谱面数据的JSON对象
            
        Returns:
            np.ndarray: (num_notes, 21) 的特征矩阵
        """
        note_groups = json_data.get('notes', [])
        if not note_groups:
            raise ValueError(f"未找到{json_data}的note group信息")
        
        # 收集所有notes数据
        all_times = []
        all_notes_data = []
        
        for note_group in note_groups:
            time = note_group['Time']
            notes = note_group['Notes']
            
            for note in notes:
                all_times.append(time)
                all_notes_data.append(note)
        
        if not all_notes_data:
            raise ValueError(f"未找到{json_data}的note信息")
        
        num_notes = len(all_notes_data)
        
        # 向量化提取所有note类型
        note_types = np.array([note['noteType'] for note in all_notes_data]).reshape(-1, 1)
        note_types_encoded = self.note_type_encoder.transform(note_types)  # (num_notes, 5)
        
        # 向量化提取所有位置
        positions = np.array([note['startPosition'] for note in all_notes_data]).reshape(-1, 1)
        positions_encoded = self.position_encoder.transform(positions)  # (num_notes, 8)
        
        # 向量化提取其他特征
        times_array = np.array(all_times, dtype=np.float32)  # (num_notes,)
        hold_times = np.array([note.get('holdTime', 0) for note in all_notes_data], dtype=np.float32)
        is_break = np.array([int(note['isBreak']) for note in all_notes_data], dtype=np.float32)
        is_ex = np.array([int(note['isEx']) for note in all_notes_data], dtype=np.float32)
        is_slide_break = np.array([int(note['isSlideBreak']) for note in all_notes_data], dtype=np.float32)
        slide_start_times = np.array([note['slideStartTime'] for note in all_notes_data], dtype=np.float32)
        slide_times = np.array([note['slideTime'] for note in all_notes_data], dtype=np.float32)
        slide_end_times = slide_start_times + slide_times
        touch_areas = np.array([self.touch_area_mapping[note['touchArea']] for note in all_notes_data], dtype=np.float32)
        
        # 组合所有特征 - 向量化拼接
        feature_matrix = np.column_stack([
            times_array,           # (num_notes, 1)
            note_types_encoded,    # (num_notes, 5)
            positions_encoded,     # (num_notes, 8)
            hold_times,            # (num_notes, 1)
            is_break,              # (num_notes, 1)
            is_ex,                 # (num_notes, 1)
            is_slide_break,        # (num_notes, 1)
            slide_start_times,     # (num_notes, 1)
            slide_end_times,       # (num_notes, 1)
            touch_areas            # (num_notes, 1)
        ])  # 总共 (num_notes, 21)
        
        return feature_matrix

    def __getitem__(self, index):
        # 从CSV中获取第index行的数据
        row = self.labels_data.iloc[index]
        json_filename = row['json_filename']
        difficulty_constant = float(row['difficulty_constant'])
        
        # 构建JSON文件的完整路径
        json_file_path = os.path.join(self.serialized_dir, json_filename)
        
        # 检查文件是否存在
        if not os.path.exists(json_file_path):
            raise FileNotFoundError(f"JSON文件不存在: {json_file_path}")
        
        with open(json_file_path, 'r', encoding='utf-8') as f:
            try:
                json_data = json.load(f)
            except json.JSONDecodeError as e:
                raise ValueError(f"JSON解析失败: {json_file_path}") from e

        # 使用向量化方法提取谱面特征序列
        note_features_matrix = self._extract_sequence_features_vectorized(json_data)

        # 将谱面数据转换为张量
        note_features_tensor = torch.from_numpy(note_features_matrix)
        difficulty_constant_tensor = torch.tensor(difficulty_constant, dtype=torch.float32)
        return note_features_tensor, difficulty_constant_tensor

    def __len__(self):
        return len(self.labels_data)

### 序列处理：
- **序列长度标准化**：使用 padding 或截断将所有序列调整为相同长度
- **序列归一化**：对时间特征进行归一化处理（暂不处理）
- **序列排序**：确保 note 按时间顺序排列（好像不需要）

**TODO**：
- 设计 note 特征的编码方案
- 确定最佳的序列长度
- 实现序列预处理管道
- 考虑是否需要添加全局特征（如 BPM、总时长等）

我们已经在自定义数据集中完成了note属性编码。


#### 特征列含义（共21列）:
```
  列 0: 时间戳 (time)
  列 1: Note类型-Tap
  列 2: Note类型-Hold
  列 3: Note类型-Slide
  列 4: Note类型-Touch
  列 5: Note类型-TouchHold
  列 6: 位置-1
  列 7: 位置-2
  列 8: 位置-3
  列 9: 位置-4
  列10: 位置-5
  列11: 位置-6
  列12: 位置-7
  列13: 位置-8
  列14: hold_time
  列15: is_break
  列16: is_ex
  列17: is_slide_break
  列18: slide_start_time
  列19: slide_end_time
  列20: touch_area
```

### 3.2 性能测试和验证

让我们测试向量化实现的性能提升，并验证结果的正确性：

In [46]:
import time

class MaichartDatasetOld(Dataset):
    """保留原始实现用于性能对比"""
    def __init__(self, serialized_dir, labels_csv):
        self.serialized_dir = serialized_dir
        self.labels_data = pd.read_csv(labels_csv)
        self.labels_data = self.labels_data.dropna(subset=['song_id', 'level_index', 'difficulty_constant', 'json_filename'])
        self.labels_data = self.labels_data.reset_index(drop=True)
        self.touch_area_mapping = {" ": 0, "A": 1, "D": 2, "E": 3, "B": 4, "C": 5}
        self._setup_encoders()

    def _setup_encoders(self):
        self.NOTE_TYPES = ['Tap', 'Hold', 'Slide', 'Touch', 'TouchHold']
        self.note_type_encoder = OneHotEncoder(sparse_output=False, dtype=np.float32, handle_unknown='ignore')
        self.note_type_encoder.fit(np.array(self.NOTE_TYPES).reshape(-1, 1))
        
        self.positions = list(range(1, 9))
        self.position_encoder = OneHotEncoder(sparse_output=False, dtype=np.float32, handle_unknown='ignore')
        self.position_encoder.fit(np.array(self.positions).reshape(-1, 1))

    def _extract_note_features(self, note, time):
        """原始的单个note特征提取方法"""
        note_type_encoded = self.note_type_encoder.transform([[note['noteType']]])[0]
        position_encoded = self.position_encoder.transform([[note['startPosition']]])[0]
        
        hold_time = note.get('holdTime', 0)
        is_break = int(note['isBreak'])
        is_ex = int(note['isEx'])
        is_slide_break = int(note['isSlideBreak'])
        slide_start_time = note['slideStartTime']
        slide_end_time = slide_start_time + note['slideTime']
        touch_area = self.touch_area_mapping[note['touchArea']]
        
        feature_vector = np.concatenate([
            [time], note_type_encoded, position_encoded,
            [hold_time], [is_break], [is_ex], [is_slide_break],
            [slide_start_time], [slide_end_time], [touch_area]
        ])
        return feature_vector

    def _extract_sequence_features_old(self, json_data):
        """原始的循环实现"""
        note_groups = json_data.get('notes', [])
        note_features_sequence = []
        
        for note_group in note_groups:
            time = note_group['Time']
            notes = note_group['Notes']
            
            for note in notes:
                feature_vector = self._extract_note_features(note, time)
                note_features_sequence.append(feature_vector)
        
        return note_features_sequence

def performance_comparison_test():
    """对比原始方法和向量化方法的性能"""
    base_dir = os.path.dirname(os.path.abspath(''))
    serialized_dir = os.path.join(base_dir, "data", "serialized")
    csv_path = os.path.join(base_dir, "data", "song_info.csv")
    
    # 创建两个数据集实例
    dataset_new = MaichartDataset(serialized_dir, csv_path)
    dataset_old = MaichartDatasetOld(serialized_dir, csv_path)
    
    # 选择测试样本
    test_indices = list(range(min(10, len(dataset_new))))  # 测试前10个样本
    
    print("性能对比测试开始...")
    print(f"测试样本数量: {len(test_indices)}")
    
    # 测试原始方法
    start_time = time.time()
    old_results = []
    for idx in test_indices:
        try:
            row = dataset_old.labels_data.iloc[idx]
            json_filename = row['json_filename']
            json_file_path = os.path.join(serialized_dir, json_filename)
            
            with open(json_file_path, 'r', encoding='utf-8') as f:
                json_data = json.load(f)
            
            features = dataset_old._extract_sequence_features_old(json_data)
            old_results.append(np.array(features))
        except Exception as e:
            print(f"原始方法处理样本 {idx} 时出错: {e}")
            continue
    
    old_time = time.time() - start_time
    
    # 测试向量化方法
    start_time = time.time()
    new_results = []
    for idx in test_indices:
        try:
            row = dataset_new.labels_data.iloc[idx]
            json_filename = row['json_filename']
            json_file_path = os.path.join(serialized_dir, json_filename)
            
            with open(json_file_path, 'r', encoding='utf-8') as f:
                json_data = json.load(f)
            
            features = dataset_new._extract_sequence_features_vectorized(json_data)
            new_results.append(features)
        except Exception as e:
            print(f"向量化方法处理样本 {idx} 时出错: {e}")
            continue
    
    new_time = time.time() - start_time
    
    # 性能结果
    print(f"\n性能对比结果:")
    print(f"原始方法耗时: {old_time:.4f} 秒")
    print(f"向量化方法耗时: {new_time:.4f} 秒")
    print(f"加速比: {old_time/new_time:.2f}x")
    
    # 验证结果正确性
    print(f"\n正确性验证:")
    if len(old_results) == len(new_results):
        all_close = True
        for i, (old_feat, new_feat) in enumerate(zip(old_results, new_results)):
            if not np.allclose(old_feat, new_feat, rtol=1e-5):
                print(f"样本 {i} 结果不一致!")
                print(f"  原始方法形状: {old_feat.shape}")
                print(f"  向量化方法形状: {new_feat.shape}")
                all_close = False
        
        if all_close:
            print("✓ 所有测试样本的结果完全一致!")
        else:
            print("✗ 发现结果不一致的样本")
    else:
        print(f"✗ 处理成功的样本数量不一致: 原始={len(old_results)}, 向量化={len(new_results)}")

# 运行性能测试
performance_comparison_test()

性能对比测试开始...
测试样本数量: 10

性能对比结果:
原始方法耗时: 0.9314 秒
向量化方法耗时: 0.0293 秒
加速比: 31.74x

正确性验证:
✓ 所有测试样本的结果完全一致!


### 3.3 向量化优化要点总结



**主要优化策略**：

1. **批量编码代替逐个编码**：
   - 原始：对每个note分别调用`OneHotEncoder.transform([[value]])`
   - 优化：收集所有note数据，一次性调用`OneHotEncoder.transform(all_values)`
   - 效果：减少了大量的函数调用开销

2. **向量化数组操作**：
   - 原始：使用Python循环和`np.concatenate`逐个拼接特征
   - 优化：使用`np.column_stack`一次性拼接所有特征列
   - 效果：利用NumPy的C语言底层实现，大幅提升性能

3. **内存访问优化**：
   - 原始：多次小数组的创建和拼接
   - 优化：预先分配大数组，减少内存分配次数
   - 效果：更好的内存局部性和缓存命中率

4. **减少中间变量**：
   - 原始：每个note创建一个中间`feature_vector`
   - 优化：直接构建最终的特征矩阵
   - 效果：减少内存开销和垃圾回收压力

**预期性能提升**：
- 对于包含大量notes的谱面，预期可获得 **5-20倍** 的性能提升
- 实际提升幅度取决于谱面的note密度和硬件配置

**兼容性保证**：
- 输出结果与原始方法完全一致
- 可直接替换原有实现，无需修改下游代码

### 3.4 Collate_fn

In [47]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    """
    自定义的collate_fn，用于处理变长序列。
    - 对note序列进行padding，使其在batch内长度一致。
    - 将标签堆叠成一个tensor。
    """
    # 1. 分离序列和标签
    # batch中的每个元素是 (note_features_tensor, difficulty_constant_tensor)
    sequences, labels = zip(*batch)

    # 2. 对序列进行padding
    # pad_sequence期望一个tensor列表
    # batch_first=True使输出的形状为 (batch_size, sequence_length, feature_dim)
    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0.0)

    # 3. 将标签堆叠成一个tensor
    # torch.stack(labels) 会创建一个 [batch_size] 的1D张量
    # .view(-1, 1) 将其转换为 [batch_size, 1] 以匹配模型输出
    labels_tensor = torch.stack(labels).view(-1, 1)

    return padded_sequences, labels_tensor

def collate_fn_with_stats(batch):
    """
    带统计信息的collate_fn，用于分析和调试。
    
    返回：
    - padded_sequences: 填充后的序列
    - labels: 标签
    - batch_stats: 包含批次统计信息的字典
    """
    if not batch:
        raise ValueError("batch为0")
    sequences, labels = zip(*batch)
    
    # 收集统计信息
    seq_lengths = [seq.size(0) for seq in sequences]
    stats = {
        'batch_size': len(sequences),
        'min_seq_length': min(seq_lengths),
        'max_seq_length': max(seq_lengths),
        'avg_seq_length': sum(seq_lengths) / len(seq_lengths),
        'total_notes': sum(seq_lengths),
        'padding_ratio': 0  # 将在padding后计算
    }
    
    # Padding
    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0.0)
    labels_tensor = torch.stack(labels).view(-1, 1)
    
    # 计算padding比例
    total_elements = padded_sequences.numel()
    padding_elements = (padded_sequences == 0).sum().item()
    stats['padding_ratio'] = padding_elements / total_elements if total_elements > 0 else 0
    
    return padded_sequences, labels_tensor, stats

### 3.6 序列长度分布分析

**问题发现**: 平均padding比例高达92.1%，这表明数据中存在序列长度极度不均匀的问题。

**可能原因**:
1. **数据中包含极长的序列**：少数极长谱面导致整体padding过多
2. **特征维度错误**：实际特征维度与预期不符
3. **数据处理错误**：序列提取过程中可能存在问题

**解决方案**:
1. 分析序列长度分布，找出异常值
2. 使用动态批处理策略
3. 考虑序列截断或分段处理

In [48]:
def analyze_sequence_lengths():
    """
    分析数据集中序列长度的分布，找出padding比例过高的原因
    """
    import matplotlib.pyplot as plt
    
    base_dir = os.path.dirname(os.path.abspath(''))
    serialized_dir = os.path.join(base_dir, "data", "serialized")
    csv_path = os.path.join(base_dir, "data", "song_info.csv")
    
    dataset = MaichartDataset(serialized_dir, csv_path)
    
    print("=== 序列长度分布分析 ===")
    print(f"数据集总大小: {len(dataset)}")
    
    # 收集序列长度统计
    sequence_lengths = []
    feature_dims = []
    sample_count = min(100, len(dataset))  # 分析前100个样本
    
    print(f"分析前 {sample_count} 个样本...")
    
    for i in range(sample_count):
        try:
            note_features, difficulty = dataset[i]
            seq_len = note_features.shape[0]
            feat_dim = note_features.shape[1] if len(note_features.shape) > 1 else 0
            
            sequence_lengths.append(seq_len)
            feature_dims.append(feat_dim)
            
            if i < 10:  # 显示前10个样本的详细信息
                print(f"  样本 {i}: 序列长度={seq_len}, 特征维度={feat_dim}, 难度={difficulty:.2f}")

            if i < 3: # 显示前3个样本的特征矩阵
                print(f"  样本 {i} 特征矩阵:\n{note_features.numpy()}")
                
        except Exception as e:
            print(f"  样本 {i} 处理出错: {e}")
            sequence_lengths.append(0)
            feature_dims.append(0)
    
    # 统计分析
    sequence_lengths = np.array(sequence_lengths)
    feature_dims = np.array(feature_dims)
    
    print(f"\n序列长度统计:")
    print(f"  最小长度: {np.min(sequence_lengths)}")
    print(f"  最大长度: {np.max(sequence_lengths)}")
    print(f"  平均长度: {np.mean(sequence_lengths):.1f}")
    print(f"  中位数长度: {np.median(sequence_lengths):.1f}")
    print(f"  标准差: {np.std(sequence_lengths):.1f}")
    
    print(f"\n特征维度统计:")
    print(f"  特征维度: {np.unique(feature_dims)}")


    
    # 计算不同批次大小的padding比例
    batch_sizes = [4, 8, 16, 32]
    print(f"\n不同批次大小的padding分析:")
    
    for batch_size in batch_sizes:
        total_padding_ratio = 0
        num_batches = 0
        
        for i in range(0, len(sequence_lengths), batch_size):
            batch_lengths = sequence_lengths[i:i+batch_size]
            if len(batch_lengths) == 0:
                continue
                
            max_len = np.max(batch_lengths)
            total_elements = len(batch_lengths) * max_len
            actual_elements = np.sum(batch_lengths)
            
            if total_elements > 0:
                padding_ratio = 1 - (actual_elements / total_elements)
                total_padding_ratio += padding_ratio
                num_batches += 1
        
        avg_padding = total_padding_ratio / num_batches if num_batches > 0 else 0
        print(f"  批次大小 {batch_size}: 平均padding比例 {avg_padding:.3f}")
    
    # 找出异常长的序列
    print(f"\n异常长序列分析:")
    percentile_95 = np.percentile(sequence_lengths, 95)
    percentile_99 = np.percentile(sequence_lengths, 99)
    
    print(f"  95%分位数: {percentile_95:.1f}")
    print(f"  99%分位数: {percentile_99:.1f}")
    
    long_sequences = sequence_lengths[sequence_lengths > percentile_95]
    print(f"  超过95%分位数的序列数量: {len(long_sequences)}")
    print(f"  这些序列长度: {sorted(long_sequences)}")
    
    return sequence_lengths, feature_dims

# 运行分析
seq_lengths, feat_dims = analyze_sequence_lengths()

=== 序列长度分布分析 ===
数据集总大小: 5478
分析前 100 个样本...
  样本 0: 序列长度=88, 特征维度=21, 难度=5.00
  样本 0 特征矩阵:
[[ 3.2  0.   0.   1.   0.   0.   0.   1.   0.   0.  ...  0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 3.2  0.   0.   1.   0.   0.   0.   0.   0.   0.  ...  0.   1.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 4.8  0.   0.   1.   0.   0.   1.   0.   0.   0.  ...  0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 4.8  0.   0.   1.   0.   0.   0.   0.   0.   0.  ...  0.   0.   1.   0.   0.   0.   0.   0.   0.   0. ]
 [ 6.4  0.   0.   1.   0.   0.   0.   0.   1.   0.  ...  0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 6.4  0.   0.   1.   0.   0.   0.   0.   0.   0.  ...  1.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 8.   0.   0.   1.   0.   0.   0.   0.   0.   1.  ...  0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 8.   0.   0.   1.   0.   0.   0.   0.   0.   0.  ...  0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 9.6  1.   0.   0.   0.   0.   0.   0.   1.   0.  

In [49]:
# 测试优化后的数据加载器
print("测试优化后的数据加载器...")

base_dir = os.path.dirname(os.path.abspath(''))
serialized_dir = os.path.join(base_dir, "data", "serialized")
csv_path = os.path.join(base_dir, "data", "song_info.csv")

# 创建优化后的数据集
optimized_dataset = MaichartDataset(serialized_dir, csv_path)

# 创建数据加载器
optimized_data_loader = DataLoader(
    optimized_dataset,
    batch_size=10,
    shuffle=False,  # 设为False以便验证结果
    collate_fn=collate_fn,
    num_workers=0
)

# 测试向量化数据加载器的性能
def test_optimized_data_loader(data_loader, num_batches=2):
    """测试优化后的DataLoader"""
    print(f"开始测试优化后的数据加载器（{num_batches} 个批次）...")
    
    start_time = time.time()
    total_notes = 0
    
    for batch_idx, (padded_sequences, labels) in enumerate(data_loader):
        print(f"\nBatch {batch_idx}:")
        print(f"  padded_sequences.shape: {padded_sequences.shape}")
        print(f"  labels.shape: {labels.shape}")
        print(f"  每个序列的note数量: {[int((seq != 0).any(dim=1).sum()) for seq in padded_sequences]}")
        
        # 统计总note数
        batch_notes = sum([int((seq != 0).any(dim=1).sum()) for seq in padded_sequences])
        total_notes += batch_notes
        print(f"  本批次总note数: {batch_notes}")
        
        # 验证特征维度
        feature_dim = padded_sequences.shape[-1]
        print(f"  特征维度: {feature_dim}")
        
        if batch_idx + 1 >= num_batches:
            break
    
    elapsed_time = time.time() - start_time
    print(f"\n性能统计:")
    print(f"  总耗时: {elapsed_time:.4f} 秒")
    print(f"  总note数: {total_notes}")
    print(f"  处理速度: {total_notes/elapsed_time:.1f} notes/秒")

# 运行测试
test_optimized_data_loader(optimized_data_loader, num_batches=2)

测试优化后的数据加载器...
开始测试优化后的数据加载器（2 个批次）...

Batch 0:
  padded_sequences.shape: torch.Size([10, 283, 21])
  labels.shape: torch.Size([10, 1])
  每个序列的note数量: [88, 116, 168, 283, 76, 135, 122, 164, 185, 198]
  本批次总note数: 1535
  特征维度: 21

Batch 1:
  padded_sequences.shape: torch.Size([10, 368, 21])
  labels.shape: torch.Size([10, 1])
  每个序列的note数量: [299, 253, 368, 126, 177, 168, 147, 138, 213, 323]
  本批次总note数: 2212
  特征维度: 21

性能统计:
  总耗时: 0.0749 秒
  总note数: 3747
  处理速度: 50044.0 notes/秒


## 4. LSTM 模型构建与数据准备

构建基于 LSTM 的时序模型来处理 note 序列数据。模型将接收形状为 `(batch_size, sequence_length, feature_dim)` 的输入，输出难度定数的预测值。

**模型架构设计**：
- **输入层**：接收编码后的 note 序列
- **LSTM层**：捕捉序列中的时序依赖关系
- **全连接层**：将 LSTM 输出映射到难度预测
- **输出层**：回归输出，预测难度定数

### 4.1 定义 LSTM 模型架构

**模型设计考虑**：
- **多层 LSTM**：评估单层 vs 多层 LSTM 的效果
- **Dropout**：防止过拟合
- **Attention 机制**：突出重要的 note 序列部分

**TODO**：
- 实现基础的 LSTM 模型类
- 设计模型的超参数（hidden_size, num_layers, dropout_rate）
- 考虑添加注意力机制
- 实验不同的模型架构

In [50]:
class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(SimpleLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        # 取最后一个时间步的输出
        last_out = lstm_out[:, -1, :]
        output = self.fc(last_out)
        return output

## 5. 模型训练与优化

**训练策略**：
- **损失函数**：使用 MSE 或 MAE 损失函数（回归任务）
- **优化器**：Adam 优化器，考虑学习率调度
- **批次处理**：合理设置 batch_size 处理变长序列
- **正则化**：Dropout + L2 正则化防止过拟合

**训练监控**：
- 训练损失和验证损失曲线
- 早停机制防止过拟合
- 学习率衰减策略

**TODO**：
- 实现训练循环
- 设置验证集监控
- 实现早停和模型保存机制
- 调试序列批次处理中的 padding 问题
- 优化训练超参数

In [ ]:
def train_model(model, data_loader, num_epochs=1, learning_rate=0.001):
    """
    训练LSTM模型

    Args:
        model: LSTM模型实例
        data_loader: DataLoader实例
        num_epochs: 训练轮数
        learning_rate: 学习率
    """
    criterion = nn.MSELoss()  # 使用均方误差损失函数
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0

        for batch_idx, (padded_sequences, labels) in enumerate(data_loader):
            optimizer.zero_grad()
            outputs = model(padded_sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            if batch_idx % 10 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx}], Loss: {loss.item():.4f}")

        avg_loss = epoch_loss / len(data_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}] 完成, 平均损失: {avg_loss:.4f}")

# 测试模型训练
def test_model_training():
    """测试LSTM模型的训练过程"""
    base_dir = os.path.dirname(os.path.abspath(''))
    serialized_dir = os.path.join(base_dir, "data", "serialized")
    csv_path = os.path.join(base_dir, "data", "song_info.csv")

    # 创建数据集和数据加载器
    dataset = MaichartDataset(serialized_dir, csv_path)
    data_loader = DataLoader(
        dataset,
        batch_size=32,
        shuffle=True,
        collate_fn=collate_fn
    )

    # 定义模型参数
    input_size = 21  # 特征维度
    hidden_size = 64  # LSTM隐藏层大小
    output_size = 1  # 输出维度（难度定数）
    num_layers = 2  # LSTM层数

    # 创建模型实例
    model = SimpleLSTM(input_size, hidden_size, output_size, num_layers)

    # 开始训练模型
    train_model(model, data_loader, num_epochs=1, learning_rate=0.001)

test_model_training()

Epoch [1/2], Batch [0], Loss: 95.0216
Epoch [1/2], Batch [10], Loss: 59.6743
Epoch [1/2], Batch [20], Loss: 35.7750
Epoch [1/2], Batch [30], Loss: 34.5730
Epoch [1/2], Batch [40], Loss: 24.1483
Epoch [1/2], Batch [50], Loss: 21.4599
Epoch [1/2], Batch [60], Loss: 23.2256
Epoch [1/2], Batch [70], Loss: 11.5874
Epoch [1/2], Batch [80], Loss: 14.4881
Epoch [1/2], Batch [90], Loss: 15.6694
Epoch [1/2], Batch [100], Loss: 15.9063
Epoch [1/2], Batch [110], Loss: 8.7980
Epoch [1/2], Batch [120], Loss: 17.4870
Epoch [1/2], Batch [130], Loss: 9.6533
Epoch [1/2], Batch [140], Loss: 15.8760
Epoch [1/2], Batch [150], Loss: 12.2290
Epoch [1/2], Batch [160], Loss: 12.1949
Epoch [1/2], Batch [170], Loss: 10.5059
Epoch [1/2], Batch [180], Loss: 14.6460
Epoch [1/2], Batch [190], Loss: 15.4255
Epoch [1/2], Batch [200], Loss: 16.5977
Epoch [1/2], Batch [210], Loss: 12.4710
Epoch [1/2], Batch [220], Loss: 16.8045
Epoch [1/2], Batch [230], Loss: 19.0256
Epoch [1/2], Batch [240], Loss: 11.8946
Epoch [1/2], 

KeyboardInterrupt: 

## 6. 模型评估与性能分析

**评估指标**：
- **回归指标**：MSE, MAE, R²
- **难度区间准确性**：预测值在真实值 ±0.1, ±0.2, ±0.5 范围内的比例
- **分布分析**：预测值与真实值的分布对比

**详细分析**：
- **不同难度等级的预测准确性**：分析模型在低难度 vs 高难度谱面上的表现
- **序列长度影响**：分析谱面长度对预测准确性的影响
- **错误案例分析**：找出预测偏差较大的谱面特征

**TODO**：
- 实现全面的评估指标计算
- 可视化预测结果分布
- 分析不同难度区间的预测准确性
- 进行错误案例的深入分析
- 与传统特征工程方法进行对比

In [ ]:
# model.eval()
# with torch.no_grad():
#     predictions = model(X_test_tensor)
#     test_loss = criterion(predictions, y_test_tensor)
#     print(f'Test Loss: {test_loss.item():.4f}')
#
# # 可以在这里添加更详细的评估指标，例如 MAE, R^2 等

## 7. 结果分析与模型迭代

**深度分析**：
- **时序特征的重要性**：LSTM 是否有效捕捉了时序信息
- **不同 note 类型的影响**：哪些类型的 note 对难度预测更重要
- **序列长度 vs 准确性**：最优的序列长度设置
- **模型复杂度 vs 性能**：单层 vs 多层 LSTM 的权衡

**模型优化方向**：
- **架构改进**：考虑 Transformer、CNN-LSTM 混合架构
- **特征增强**：是否需要添加手工特征作为辅助
- **数据增强**：通过时间扭曲、音符变换等方式增加训练数据
- **多任务学习**：同时预测难度和其他属性（如技巧需求）

**TODO**：
- 深入分析 LSTM 学到的时序模式
- 可视化注意力权重（如果使用了注意力机制）
- 比较不同模型架构的效果
- 设计更鲁棒的数据增强策略
- 考虑集成学习方法提升性能
- 为生产环境部署准备模型压缩和优化

分析模型的预测结果，与真实定数进行比较。

思考以下问题：
- 模型的误差主要来自哪些谱面？
- 是否有必要调整特征工程的方案？
- 是否需要更复杂的模型结构？

根据分析结果，回到前面的步骤进行迭代优化。

**关键思考问题**：

1. **时序建模的有效性**：
   - LSTM 是否真的比传统统计特征更有效？
   - 谱面的时序特征对难度的影响有多大？

2. **数据表示的完整性**：
   - 当前的 note 编码是否充分表达了游戏的复杂性？
   - 是否遗漏了重要的游戏机制信息？

3. **模型的可解释性**：
   - 如何理解模型学到的难度判断规律？
   - 能否提取出可解释的难度评估规则？

4. **实际应用价值**：
   - 模型的预测精度是否满足实际需求？
   - 如何将模型集成到谱面制作工具中？

**下一步迭代方向**：
根据实验结果，有针对性地改进数据处理、模型架构或训练策略，最终目标是构建一个既准确又实用的难度预测系统。